# Face Recognize in Video using FaceNET, OpenCV

Imports Libraries

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
import os
import cv2
import time
import pickle
import imutils
import argparse
import numpy as np
from imutils.video import VideoStream
from imutils.video import FPS
from tensorflow.keras.models import load_model

Argument Parser

In [ ]:
ap=argparse.ArgumentParser()

ap.add_argument("--video",required=True,help="input video path")
ap.add_argument("--faceModel",required=True,help="path for face detection model")
ap.add_argument("--prototxt",required=True,help="path of prototxt model")
ap.add_argument("--embeddingModel",required=True,help="path of embedding model")
ap.add_argument("--recognizer",required=True,help="path of train SVM model")
ap.add_argument("--le",required=True,help="path to label encoder")

args=vars(ap.parse_args(["--video",r"C:\Users\SRKT\Desktop\opencv-face-recognition\output\4.mp4",
                        "--faceModel",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\face_detection_model\res10_300x300_ssd_iter_140000.caffemodel",
                        "--prototxt",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\face_detection_model\deploy.prototxt",
                        "--embeddingModel",r"C:\Users\SRKT\Desktop\opencv-face-recognition\pre\openface_nn4.small2.v1.t7",
                        "--recognizer",r"C:\Users\SRKT\Desktop\opencv-face-recognition\output\Test\recognizer2.h5",
                        "--le",r"C:\Users\SRKT\Desktop\opencv-face-recognition\output\label.pickle"
                        ]))

Loading Model

In [ ]:
#loading face detection model
face_detector=cv2.dnn.readNetFromCaffe(args["prototxt"],args["faceModel"])

#loading face embedding model
face_embedder=cv2.dnn.readNetFromTorch(args["embeddingModel"])

#loading the actual face recognition model 
face_recognizer=load_model(args["recognizer"])

#label encoder loading
le=pickle.loads(open(args["le"],"rb").read())

Initializing Video Stream ,and detect by frame by frame in video

In [ ]:
video=cv2.VideoCapture(r"C:\Users\SRKT\Desktop\opencv-face-recognition\output\4.mp4")
#video=VideoStream(args["video"]).start()
#fps=FPS().start()
#loop over frame by frame
while True:
    ret,frame=video.read()
    
    #resize the frame and then grab the image dimenssions
    frame=cv2.resize(frame,(650,600))
    (h,w)=frame.shape[:2]
    
    #construct the blob from image
    imageBlob=cv2.dnn.blobFromImage(frame,1.0,(300,300),(104.0,177.0,123.0),swapRB=False,crop=False)
    
    #apply OpenCV deep learning based face detector to localize faces in the input image
    face_detector.setInput(imageBlob)
    detections=face_detector.forward()
    
    #loop over the detections
    for i in range(0,detections.shape[2]):
        #extract the confidence (i.e probability) associate with the predictions
        confidence=detections[0,0,i,2]
        
        #filter out the weak detections
        if confidence>0.5:
            #compute the (x,y) coordinate of the bounding box of the faces
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (start_x,start_y,end_x,end_y)=box.astype("int")
            
            #extract the face ROI
            face=frame[start_y:end_y,start_x:end_x]
            (fH,fW)=face.shape[:2]
            
            #ensure face width & height are sufficently large
            if fW<20 or fH<20:
                continue
            
            #construct the blob for the face ROI , then pass the blob through our face embedding model
            #to obtain the 128-d quantifications of faces 
            faceBlob=cv2.dnn.blobFromImage(face,1.0/255,(96,96),(0,0,0),swapRB=True,crop=False)
            face_embedder.setInput(faceBlob)
            vec=face_embedder.forward()
            
            #perform the classifications to recognize the face
            preds=face_recognizer.predict_proba(vec)[0]
            j=np.argmax(preds)
            proba=preds[j]
            name=le.classes_[j]
            
            #draw the bounding box of the face along the associate probability
            text="{} : {:.2f}%".format(name,proba*100)
            y=start_y-10 if start_y-10>10 else start_y+10
            cv2.rectangle(frame,(start_x,start_y),(end_x,end_y),(0,0,255),2)
            cv2.putText(frame,text,(start_x,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,(0,0,255),2)
           
    #fps.update()
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) &0xFF
    if key==ord("q"):
        break
#fps.stop()
cv2.destroyAllWindows()
video.stop()